# Lab Sheet 6 (COM3502-4502-6502 Speech Processing)

This lab sheet is part of the lecture COM[3502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/level3/com3502.html "Open web page for COM3502 module")-[4502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/level4/com4502.html "Open web page for COM4502 module")-[6502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/msc/com6502.html "Open web page for COM4502 module") Speech Processing at the [University of Sheffield](https://www.sheffield.ac.uk/ "Open web page of The University of Sheffield"), Dept. of [Computer Science](https://www.sheffield.ac.uk/dcs "Open web page of Department of Computer Science, University of Sheffield").

It is probably easiest to open this Jupyter Notebook with [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true "Open in Google Colab") since GitHub's Viewer does not always show all details correctly. <a href="https://colab.research.google.com/github/sap-shef/SpeechProcesssingLab/blob/main/Lab-Sheets/Lab-Sheet-6.ipynb"><img align="right" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open Notebook in Google Colab" title="Open and Execute the Notebook directly in Google Colaboratory"></a>

Please put questions, comments and correction suggestions in the [Blackboard](https://vle.shef.ac.uk) discussion board or send an email to [s.goetze@sheffield.ac.uk](mailto:s.goetze@sheffield.ac.uk).

In [ ]:
# Let's do the ususal necessary and nice-to-have imports
%matplotlib inline
import matplotlib.pyplot as plt  # plotting
import seaborn as sns; sns.set() # styling ((un-)comment if you want)
import numpy as np               # math

# imports we need in addition for this lab sheet
from IPython import display as ipd
import scipy.signal as sig

## The Chirp Signal, a.k.a. Sweep Signal

The function 
\begin{equation}
x_{\mathrm{chirp}}(t)=\mathrm{sin}\left(\pi t^2\right) \tag{1}
\end{equation}
defines the so-called **linear-frequency [chirp](https://en.wikipedia.org/wiki/Chirp "Click here for additional information on the Chirp signal on Wikipedia")** or simply linear chirp. It is a signal that sweeps through diffrent frequencies over time.


<br>
<a id='task_1'></a>
<div style="border: 2px solid #999; padding: 10px; background: #eee;">
    
**Task 1: Create and analyse a Chirp Signal**
    
<ul>
<li> Implement a function <code>generate_chirp</code> that outputs a sampled chirp signal <code>x</code> for the time interval $[t_0,t_1]$ in seconds with $0\leq t_0<t_1$.
</li>     
<li>
    Compute the the chirp signal <code>x</code> for various input parameters $t_0$ and $t_1$. Hint: also try compatatively high <code>t</code>'s, e.g. <code>t1=50</code>, <code>t2=200</code>.
</li>
<li> 
    Visulaise and listen to the generated signal(s). Please be aware that for low <code>t</code> you also create a signal with low frequency $f$. Your headphones / loudspeakers might not be capable to reprocude signals with such low frequencies. It should, however, work for higher $t$ and $f$.
</li> 
<li>
   Calculate and visualise the magnitude Fourier transform ($|\mathrm{DFT}(x)|$).
</li>
</ul>
</div>

**Hint**: For finding a proper [FFT](https://en.wikipedia.org/wiki/Fast_Fourier_transform) length $L_{\mathrm{FFT}}$ we are interested in a number which is higher than the length of our time domain sequence. Furthermore, a power of 2 is beneficial to allow the use of the efficient [Radix-2 implementation](https://en.wikipedia.org/wiki/Cooley%E2%80%93Tukey_FFT_algorithm) for the `fft()`. The following function `nextPowerOf2(L)` calculates an appropriate FFT length `L_FFT` which is a power of $2$ and is always longer than the signal length $L$. Play around with different $L$, e.g. by using the example provided in the main comment in the code below.

```python
for L in range(20):
    print('nextPowerOf2(L) for L='+str(L)+' is '+str(nextPowerOf2(L)))
```

In [ ]:
def nextPowerOf2(L):
    '''
    Calculates the smallest power of 2 which is bigger than input variable n
    
    This helper function can be used to calculate an appropriate 
    length for an DFT which is longer than the signal length n and a power of 2.
    
    Input:
        L: int
            signal length
    Output:
        p: integer which is greater or equal than n and a power of 2
    
    Example:
        for L in range(20):
            print('nextPowerOf2(L) for L='+str(L)+' is '+str(nextPowerOf2(L)))
    '''
    if (L<2):
        return 2
    # If n is a power of 2 then return n 
    if (L and not(L & (L - 1))):
        return L
    # If n is NOT a power of 2 
    p = 1
    while (p < L) :
        p <<= 1 
    return p

Complete the code below to solve Task 1.

In [ ]:
fs=8000      # sampling frequency
#t1=???         # time 1 in seconds
#t2=???        # time 2 in seconds
length=t2-t1 # signlal length in seconds

# create chirp signal
# ...

# listen chirp signal
# ...

# plot chirp signal
# ...

# plot spectrum of chirp signal
# ...

The `scipy.signal` library provides a function `chirp()` which is more flexible than the function we created before. See e.g. [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.chirp.html) for a documentation. 

<br>
<a id='task_2'></a>
<div style="border: 2px solid #999; padding: 10px; background: #eee;">
    
**Task 2**
    
<ul>
<li>
    Create a linear chirp of $10$ seconds length which has a frequency of $f_0$ = $10$ Hz at the time $t=0$ seconds and $f_1$ = $6$ Hz at the time $t=5$ seconds. 
</li>
    <li>
    Play around with the frequencies and also generate a <strong>logarithmic chirp</strong> (see documentation link above).
    </li>
</ul>
</div>


In [ ]:
import scipy.signal as sig

fs=8000   # sampling frequency
length=10 # signlal length in seconds

f0 = 10         # frequency in Hz at time t_0=0
f1 = 6          # frequency in Hz at time t_1
t1 = 5          # frequency in Hz at time t=0

# t =
# chp1 = sig.chirp(# ....

# Your code here
# ...

### Pyplot Spectrogram vs. Spectrum of Chirp Signal

The spectrum calculated in the previous task only shows the mean spectrum (averaged over time) which does not fully reflect what we perceived while listening to the signal.

<br>
<a id='task_3'></a>
<div style="border: 2px solid #999; padding: 10px; background: #eee;">
    
**Task 3: Sectrogram of Chirp Signal**
    
<ul>
    <li> 
        Show the spectrogram of the created chirp signal using the <code>plt.specgram()</code> function and compare it to the previously calculated spectra.
    </li>
</ul>
</div>

In [ ]:
# Your code here
# ...

## Short-Time Fourier Transform

### Some Background Theory

The discrete Fourier transform (DFT) is not very well suited for the analysis of instationary signals (such as an speech or music signal or the chirp signal) since it is applied to the entire signal at once. Thus, the mean spectum is calculated in this case which might not be very insightful. 

To gain more insight in changes of the spectrum over time we split a long signal into segments and compute the DFT for each of these segments. This is known as the [short-time Fourier transform](https://en.wikipedia.org/wiki/Short-time_Fourier_transform) (STFT).

The STFT $x[n,\ell]$ of a time domain signal $x[k]$ is defined as

\begin{equation}
x[n, \ell] = \sum_{k = \ell L_{\mathrm{hop}}}^{\ell L_{\mathrm{hop}}+L_{\mathrm{DFT}}-1} x[k] \, w[k+\ell L_{\mathrm{hop}}] \;  \mathrm{e}^{\,-\mathrm{j}\,\frac{2 \pi}{L_{\mathrm{DFT}}} k n}
\end{equation}

FOR EVERY BLOCK $\ell$, where $w[k]$ denotes a window function of length $L_w$ (often of same length as the DFT length, i.e. $L_w=L_{\mathrm{DFT}}$) which is normalized by $\sum_{k=0}^{L_w-1} w[k] = 1$. Starting from $k=\ell L_{\mathrm{hop}}$, the signal $x[k]$ is windowed by $w[k]$ to a segment of length $L_w$. This windowed segment is then transformed by a DFT of length $L_{\mathrm{DFT}}$.

The STFT can be used for spectral analysis of signals or processing of instationary signals. The resulting spectra $x[n,\ell]$ depend on the frequency index $n$ and the block index $\ell$ which is directly related to the time index $k$ by $k=\ell L_{\mathrm{hop}}$. The spectral domain is therefore also termed as [time-frequency domain](https://en.wikipedia.org/wiki/Time%E2%80%93frequency_representation) and techniques using the STFT as time-frequency processing.

The properties of the STFT depend on

* the DFT length $L_{\mathrm{DFT}}$ of the segments,
* the overlap between the segments $L_{\mathrm{hop}}$, and
* the window function $w[k]$.

The size $L_{\mathrm{DFT}}$ of the segments and the window function influence the spectral and temporal resolution of the STFT. 

## Creating the Spectrogram Step-by-Step

The magnitude $|x[n, \ell]|$ of the STFT for all $n$ and $\ell$ is known as the [spectrogram](https://en.wikipedia.org/wiki/Spectrogram) of a signal. It is frequently used to analyze signals in the time-frequency domain, for instance by a [spectrum analyzer](https://en.wikipedia.org/wiki/Spectrum_analyzer). It can be interpreted as a *image* of the signal with (block) time direction on the $x$ axis and (discrete) frequency $n$ on the y axis.

From Lab Sheet 3 we already know how to brack a long signal into block, a.k.a. frames.

<br>
<a id='task_4'></a>
<div style="border: 2px solid #999; padding: 10px; background: #eee;">
    
**Task 4: Manual Spectrogram Calculation**
    
<ul>
    <li> 
        Split the input sequence into blocks of length $L_{\mathrm{DFT}}$. These may be overlapping. We solved this task already in Lab Sheet 3. Note: as an input signal $x[k]$ you may either want to load a WAVE file or just take a chirp signal already generated above.
    </li>
    <li> 
        Multiply every frame with a window function $w[k]$. You can create a window using the function <code>np.hamming(L)</code>. The window should have the same length $L_{\mathrm{DFT}}$ as your frame.
    </li>
    <li> 
        Transform every windowed frame to the frequency domain using the <code>fft()</code>.
    </li>
    <li> 
        Concatenate the resulting spectra to a spectrogram and display the resulting spectrogram. You can use <code>matplotlib</code>'s <code><a href="https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html">imshow()</a></code> function for manually plotting the spectrogram image. Note that a spectrogram is usally shown in dB scaling.
    </li>
</ul>
</div>

In [ ]:
# Your code here
# ...

## Copyright

This notebook is licensed to be used during the lecture COM[3502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/level3/com3502.html "Open web page for COM3502 module")-[4502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/level4/com4502.html "Open web page for COM4502 module")-[6502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/msc/com6502.html "Open web page for COM4502 module") Speech Processing at the [University of Sheffield](https://www.sheffield.ac.uk/ "Open web page of The University of Sheffield"), Dept. of [Computer Science](https://www.sheffield.ac.uk/dcs "Open web page of Department of Computer Science, University of Sheffield"). Any further use (beyond use for the lecture) is only permitted if agreed with the [module lead](mailto:s.goetze@sheffield.ac.uk). 